### Uso de Tools en Gemini

En esta práctica vamos a aprender a usar directamente la API de Gemini.

In [2]:
# Importar las librerías necesarias
import os
from typing import Any, Dict

import google.generativeai as genai
import requests
from dotenv import load_dotenv

print("¡Estamos en marcha!")

¡Estamos en marcha!


In [3]:
# Cargar las variables de entorno
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key or api_key.startswith("ADD YOUR"):
    raise ValueError("GOOGLE_API_KEY no encontrada en el archivo .env")

# Configurar Gemini
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-2.0-flash")

print(f"Modelo Gemini cargado correctamente: {model.model_name}")

Modelo Gemini cargado correctamente: models/gemini-2.0-flash


Comprobamos que no podemos conocer el precio de cotización actual de Bitcoin.

In [7]:
# Definir el mensaje de consulta
PROMPT = "¿Cuál es el precio actual de Bitcoin?"
chat = model.start_chat()
response = chat.send_message(PROMPT)
print(response.text)

El precio de Bitcoin cambia constantemente. Para obtener el precio actual más preciso, te recomiendo consultar alguna de las siguientes fuentes:

*   **CoinMarketCap:** [https://coinmarketcap.com/currencies/bitcoin/](https://coinmarketcap.com/currencies/bitcoin/)
*   **Coinbase:** [https://www.coinbase.com/price/bitcoin](https://www.coinbase.com/price/bitcoin)
*   **Binance:** [https://www.binance.com/en/price/bitcoin](https://www.binance.com/en/price/bitcoin)

Estas páginas web te mostrarán el precio en tiempo real, así como gráficos de su evolución y otra información relevante.



Podemos ver la estructura de la respuesta que nos devuelve Gemini.

In [8]:
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "El precio de Bitcoin cambia constantemente. Para obtener el precio actual m\u00e1s preciso, te recomiendo consultar alguna de las siguientes fuentes:\n\n*   **CoinMarketCap:** [https://coinmarketcap.com/currencies/bitcoin/](https://coinmarketcap.com/currencies/bitcoin/)\n*   **Coinbase:** [https://www.coinbase.com/price/bitcoin](https://www.coinbase.com/price/bitcoin)\n*   **Binance:** [https://www.binance.com/en/price/bitcoin](https://www.binance.com/en/price/bitcoin)\n\nEstas p\u00e1ginas web te mostrar\u00e1n el precio en tiempo real, as\u00ed como gr\u00e1ficos de su evoluci\u00f3n y otra informaci\u00f3n relevante.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.1

Usamos una de las recomensaciones de Gemini para obtener el precio de cotización actual de Bitcoin.

In [5]:
url = f"https://api.binance.com/api/v3/ticker/price?symbol=BTCUSDT"
response = requests.get(url)
data = response.json()
print(data)

{'symbol': 'BTCUSDT', 'price': '109177.23000000'}


Creamos una función que nos devuelve el precio de cotización actual de Bitcoin.

In [ ]:
# Definir la función
def get_crypto_price(symbol: str) -> float:
    """
    Obtener el precio actual de una criptomoneda desde la API de Binance
    """
    url = f"https://api.binance.com/api/v3/ticker/price?symbol={symbol}"
    response = requests.get(url)
    data = response.json()
    return float(data["price"])

In [10]:
precio = get_crypto_price("BTCUSDT")
print(f"Precio de BTC en USDT: {precio}")

Precio de BTC en USDT: 109303.19


Creamos una `tool` que llame a la función anterior. Los comentarios son importantes ya que no podemos suponer que Gemini conoce la API de Binance.

In [12]:
tools = [
    {
        "function_declarations": [
            {
                "name": "get_crypto_price",
                "description": "Obtener el precio de una criptomoneda en USDT desde Binance",
                "parameters": {
                    "type": "object", 
                    "properties": {
                        "symbol": {
                            "type": "string",
                            "description": "El símbolo del par de trading de la criptomoneda (por ejemplo, BTCUSDT, ETHUSDT). \
                                            El símbolo para Bitcoin es BTCUSDT. \
                                            El símbolo para Ethereum es ETHUSDT."
                        }
                    },
                    "required": ["symbol"]
                }
            }
        ]
    }
]

Hacemos la misma pregunta pero pasando a Gemini la tool que acabamos de crear. Observamos que ahora Gemini no responde con un campo `text` sino con un campo `parts`. Con esto Gemini nos está indicando que quiere usar la `tool` para obtener el precio de cotización actual de Bitcoin. Los LLMs nunca llaman directamente a las `tools`, sino que nos piden que las llamemos nosotros.

In [15]:
PROMPT = "¿Cuál es el precio actual de Bitcoin (BTCUSDT)?"
chat = model.start_chat()
response = chat.send_message(PROMPT, tools=tools)
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "get_crypto_price",
                  "args": {
                    "symbol": "BTCUSDT"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.0027632578276097775
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 72,
        "candidates_token_count": 8,
        "total_token_count": 80
      },
      "model_version": "gemini-2.0-flash"
    }),
)


Hacemos lo que nos pide Gemini.

In [16]:
price = get_crypto_price("BTCUSDT")

Y le enviamos la respuesta de la `tool` a Gemini.

In [17]:
final_response = chat.send_message(str(price))
print(final_response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "El precio actual de Bitcoin (BTCUSDT) es 109027.72.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.002813226797363975
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 29,
        "candidates_token_count": 22,
        "total_token_count": 51
      },
      "model_version": "gemini-2.0-flash"
    }),
)


In [18]:
print(final_response.text)

El precio actual de Bitcoin (BTCUSDT) es 109027.72.

